# Half Cheetah with NEAT brain

## Imports

In [11]:
import gym
import random as r
import os
import neat
import msvcrt as m
from sys import exit
#remember to start xming server by using 
#set DISPLAY=:0 
#in the command line
#Build the environment for the cheetah
env = gym.make('HalfCheetah-v2') #This is a global definition

#load the config file
config_path = os.path.join('config.txt')

## Functions to connect the brain to the environment, run the brain and assess the effectiveness

In [12]:
def connect_brain(nLimbs, nCycles, net, render = False):
    '''
    Connect brain takes arguments:
    number of limbs = 6 for cheetah sausage
    number of cycles = run time for each NN to control the cheetah
    net = the NN object
    render = flag to render the simulation to screen    
    
    Connecting the brain effectively feeds the NN input nodes with the state of the environment, 
    and feeds the environment with the output nodes of the NN
    '''
    
    #reseting the cheetah environment
    env.reset()
    
    #initialise an empty set with the number of control inputs going to mujoco
    x = [] 
    for i in range(nLimbs):
        x.append(0)
    
    #After each step of the envronment running, a result is returned that contains:
    #observation for the NN brain to see that is fed to the NN input
    #reward = the distance traveled during that cycle of the simulation with a maximum goal
    #Result needs to be initialised (with a set of controls set to 0) to be fed to the neural network
    
    result = env.step(x) 
    observation = result [0]
    reward = result [1]
    fitness = 0
    
    #control is the set of output nodes from the NN
    control = net.activate(observation) #0th observation to initialise control
    
    for i in range(nCycles): 
        #The NN is activated by giving observation to the input nodes, and storing the output nodes in control
        control = net.activate(observation)
        
        #Control needs to be set the the correct range
        #the NN outputs a float between 0 and 1
        #the environment takes a float between -1 and 1
        for j in range(len(control)):
            control[j] = control[j]*2 - 1
        
        #This loop creates a delay between the NN brain and the environment body.
        #This simulates the natural delay between signal and action and allows 
        #commands to be completed before a new command is given.
        for j in range(4): #stepping through multiple times to delay the NN
            #output of the environment is stored and split into useable data
            result = env.step(control) 
            reward = result [1]
            observation = result[0]
             
            #reward is summed to make fitness; the total distance travelled in the given number of cycles
            fitness = fitness + reward
        
            if render:
                env.render()
                
    return fitness
    
    

def new_pop(config_file):
    '''
    function to create a new population for assessment 
    config_file is used to set the initial state of the population
    '''
    
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file) 

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))
    
    return p
    
    

#This is the fitness function
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        
        limbs = len(env.action_space.sample())
        cycles = 20
        
        #Builds the neural net from the genome
        net = neat.nn.FeedForwardNetwork.create(genome, config)

        genome.fitness = connect_brain(limbs, cycles, net)

## Make a new population and run for n generations

In [13]:
#number of generations
n = 1 

p = new_pop(config_path)
p.run(eval_genomes, n)


 ****** Running generation 0 ****** 

Population's average fitness: -38.56328 stdev: 25.31871
Best fitness: 29.90996 - size: (12, 240) - species 1 - id 125
Average adjusted fitness: 0.601
Mean genetic distance 2.603, standard deviation 0.209
Population of 300 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   300     29.9    0.601     0
Total extinctions: 0
Generation time: 3.869 sec


## Load the population from a checkpoint

In [14]:
#Continue running from previous saved file

checkpoint = 'neat-checkpoint-2099'

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path) 

p = neat.Checkpointer.restore_checkpoint(checkpoint)  

# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))


## Continue running the assessment of the loaded population

In [16]:
# Run for up to n generations.
n = 1

winner = p.run(eval_genomes, n)


# Display the winning genome.
print('\nBest genome:\n{!s}'.format(p.best_genome))  


 ****** Running generation 2099 ****** 

Population's average fitness: 52.39435 stdev: 61.68184
Best fitness: 163.83488 - size: (11, 5) - species 189 - id 48
Average adjusted fitness: 0.529
Mean genetic distance 2.576, standard deviation 0.622
Population of 235 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   185   31    17    111.2    0.584     3
   187   29    36    155.8    0.575     0
   188   24    46    149.0    0.634     9
   189    7    66    163.8    0.552     0
   190    6    21    120.7    0.558     0
   191    0    36    122.2    0.530     0
   192    0     8     86.4    0.269     0
   193    0     5       --       --     0
Total extinctions: 0
Generation time: 2.174 sec
Saving checkpoint to neat-checkpoint-2099

Best genome:
Key: 48
Fitness: 163.83488429912285
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.7956297172910975, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=-1.87195096374

## Display the genome with the best fitness

In [ ]:

best = p.best_genome
winner_net = neat.nn.FeedForwardNetwork.create(best, config)
connect_brain(6, 10000, winner_net, True)
env.close()

Creating window glfw


## Test box

In [ ]:
import gym
import random as r

env = gym.make('HalfCheetah-v2') #This is a global definition to use the neural network junk
env.reset()
for i in range (5000):
    if i % 20 < 10:
        control = [1,1,1,1,1,1]
    else:
        control = [-1,-1,-1,-1,-1,-1]
    #control = [1,1,1,1,1,1]
    #control = [0,0,0,0,0,0]
    new = []
    out = env.step(control)[0]
    for j in out:
        
        j = float("{0:.2f}".format(j))
        new.append(j)
    print (new)
    env.render()